<a href="https://colab.research.google.com/github/2440042872shravan/Gitlink-exercise-week-2-2440042872shravan/blob/master/ensemble_method_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import pickle
np.random.seed(32113)


def to_prepare_data_esembler(df1,df2,df3,df4, lbl = 'word', countries=['US','BR','RU','KR'],\
                   words=['USA','Brazil','Russia','Korea'],sample=30000, limit = 5000):
    
    # if running image recognition, 
    if lbl == 'word':
      #runs _df_initial_fixer for each word to prepare dataframe
        df_test1 = _df_initial_fixer(df1,words[0],sample)
        df_test2 = _df_initial_fixer(df2,words[1],sample)
        
        print(len(df_test1))
        print(len(df_test2))
        # convining all 4 dataframe to create a new dataframe. the new_df will be the input for XGB.
        new_df = pd.concat([df_test1,df_test2], axis =0)
        yd = new_df.pop('countrycode')
        Y = new_df.pop('word')
        b_loon={}
        for i in range(len(words)):
            b_loon[words[i]] = i
            print
        # Y will be the label for my XGB model.
        Y = Y.map(b_loon)
        return new_df,Y
    
    # if running country prediction,
    elif lbl == 'countrycode':
      #runs _df_initial_fixer_cc for each word to prepare dataframe
        df_test1 = _df_initial_fixer_cc(df1,words[0])
        df_test2 = _df_initial_fixer_cc(df2,words[1])
        print(len(df_test1))
        print(len(df_test2))
        new_df = pd.concat([df_test1,df_test2], axis =0)
        
        #filter dataframe by selected countries
        df_cf = new_df[(new_df['countrycode']==countries[0])|(new_df['countrycode']==countries[1])|\
                   (new_df['countrycode']==countries[2])|(new_df['countrycode']==countries[3])]
        print(len(df_cf))

        # US
        df_US = _country_initial_fixer(df_cf,countries[0],limit)
        #BR
        df_BR = _country_initial_fixer(df_cf,countries[1],limit)
        #RU
        df_RU = _country_initial_fixer(df_cf,countries[2],limit)
        #KR
        df_KR = _country_initial_fixer(df_cf,countries[3],limit)

        print("number of images for US:{}, BR:{}, RU:{}, KR:{}\n")
        print(len(df_US))
        print(len(df_BR))
        print(len(df_RU))
        print(len(df_KR))
        # new_df will be the input Dataframe for XGBoost and Y will be the label
        new_df = pd.concat([df_US,df_BR,df_RU,df_KR], axis=0)
        Y = new_df.pop('countrycode')
        b_loon = {}
        for i in range(len(countries)):
            b_loon[countries[i]] = i
        Y = Y.map(b_loon)
       
        # creating additional feature called word. In this feature number represents the word of image.
        b_loon2={'airplane':0,'eyeglasses':1}
        new_df['word']=new_df['word'].map(b_loon2)

        return new_df,Y #,df_US,df_BR,df_RU,df_KR
    else:
        print("set your lbl to 'word' or 'countrycode'")

def _df_initial_fixer(df, word, sample=60000):

    print("total number of images for df_{}: {}").format(word, len(df))
    random_index = np.random.choice(list(df.index), sample, replace=False)
    df = df.loc[list(random_index)]
    df_test = df.drop(['drawing','key_id','timestamp','recognized','X','Y','time',\
                        'X_per_stroke','Y_per_stroke','time_per_stroke',\
                        'total_time_of_stroke','dp_per_stroke','dp_percent_per_stroke',\
                        'direction'],axis=1)
    return df_test

def _df_initial_fixer_cc(df, word):

    df_test = df.drop(['drawing','key_id','timestamp','recognized','X','Y','time',\
                        'X_per_stroke','Y_per_stroke','time_per_stroke',\
                        'total_time_of_stroke','dp_per_stroke','dp_percent_per_stroke',\
                        'direction'],axis=1)
    return df_test

def _country_initial_fixer(df,country,limit):
  
    if df[df['countrycode']==country].count()[0] > limit:
        df_c = df[df['countrycode']==country]
        random_c = np.random.choice(list(df_c.index), limit, replace=False)
        df_c = df_c.loc[list(random_c)]
    else:
        df_c = df[df['countrycode']==country]
    return df_c